In [ ]:
require(doParallel)

In [ ]:
source('01_hmm_fwdbwd.r')
source('01_hmm_loglike.r')
source('02_hmm_em.r')
source('02_shapeisequal.r')
source('10_shapesearch.r')
source('20_reduction.r')
source('30_compno_search.r')
source('40_initialization.r')

In [ ]:
load(file = 'dat.RData')

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
par(mfrow=c(1, 2))

plot(dat, main='dat', type='l', xlab='t', ylab='counts')
plot(dat_scalemodifier, main='scmod', type='l', xlab='t', ylab='')

In [ ]:
nthreads = 2
totaljobs = 4
minspread = 1
maxspread = 100

In [ ]:
base_spreads = as.integer(seq(minspread, maxspread, length=totaljobs))
base_shape = 1:4
shape_m = base_spreads %*% t(base_shape)
print (shape_m)

In [ ]:
worker = function(shape, dat, dat_scalemodifier, verbose = T) {
    t0 = proc.time()
    empars = empars_ini(shape, dat, dat_scalemodifier)
    mc_em_output = mc_em(dat, dat_scalemodifier, empars, verbose = verbose)
    out = em_compno_search(mc_em_output, dat, dat_scalemodifier, verbose = verbose)
    t1 = proc.time()
    
    if (verbose) {
        cat('shape:', shape, 'done in:', (t1-t0)['elapsed'])
    }
    
    return (out)
}

In [ ]:
worker(shape_m[1,], dat, dat_scalemodifier)

In [ ]:
cl = makePSOCKcluster(nthreads)
registerDoParallel(cl)

tstart = proc.time()
res <- foreach(i = 1:nrow(shape_m),
               .inorder = F,
               .errorhandling = 'remove',
               .export = ls(all.names=T),
               .verbose = T) %dopar% { 
    worker(shape_m[i,], dat, dat_scalemodifier)
}

print(proc.time()-tstart)
stopCluster(cl)

In [ ]:
save(res, file = 'fitres.RData')

In [ ]:
# collect all results from different spreads
all_aics = c()
all_nstates = c()

for (model in res) {
    all_aics = c(all_aics, model$aic)
    all_nstates = c(all_nstates, length(model$empars$pars_list$shapes))
}

In [ ]:
min_aic = min(all_aics)
min_aic_dex = which.min(all_aics)
cat('min aic is:', min_aic,
    'at dex:',     min_aic_dex,
    '\n')

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
par(mfrow=c(1, 2))

plot(base_spreads, all_aics, type='b', main='aic', xlab='spread', ylab='')
abline(v=base_spreads[min_aic_dex], col='red')
plot(base_spreads, all_nstates, type='b', main='nstate', xlab='spread', ylab='')
abline(v=base_spreads[min_aic_dex], col='red')

In [ ]:
bestmodel=res[[min_aic_dex]]
print (list('iniprob'=bestmodel$empars$mc_iniprob,
            'transprob'=bestmodel$empars$mc_transition,
            'shapes'=bestmodel$empars$pars_list$shapes,
            'scale'=bestmodel$empars$pars_list$scale))

In [ ]:
load('truemodel.RData')
print (truemodel)